# Introduction

## What is Terragrunt

Terragrunt is a thin wrapper that provides extra tools for keeping your configurations DRY, working with multiple Terraform modules, and managing remote state.

To use it, you:

- Install Terraform.
- Install Terragrunt.
- Put your Terragrunt configuration in a `terragrunt.hcl` file. You’ll see several example configurations shortly.
- Now, instead of running terraform directly, you run the same commands with terragrunt:

```shell
terragrunt plan
terragrunt apply
terragrunt output
terragrunt destroy
```

Terragrunt will forward almost all commands, arguments, and options directly to Terraform, but based on the settings in your terragrunt.hcl file.

## Use Cases

- Keep your Terraform code DRY
- Keep your remote state configuration DRY
- Keep your CLIE flags DRY
- Execute Terraform commands multiple modules at once
- Work with multiple AWS accounts

<details><summary><h3>Keep your code DRY</summary>

```text
live
├── prod
│   ├── app
│   │   └── terragrunt.hcl
│   ├── mysql
│   │   └── terragrunt.hcl
│   └── vpc
│       └── terragrunt.hcl
├── qa
│   ├── app
│   │   └── terragrunt.hcl
│   ├── mysql
│   │   └── terragrunt.hcl
│   └── vpc
│       └── terragrunt.hcl
└── stage
    ├── app
    │   └── terragrunt.hcl
    ├── mysql
    │   └── terragrunt.hcl
    └── vpc
        └── terragrunt.hcl
```
</details>

In [ ]:
mkdir -p live/{prod,qa,stage}/{app,mysql,vpc}
touch live/{prod,qa,stage}/{app,mysql,vpc}/terragrunt.hcl
tree live

<details><summary><h3>Keep your remote state configuration DRY</summary>

```go
generate "remote_state" {
  path      = "backend.tf"
  if_exists = "overwrite"
  contents = <<EOF
terraform {
  backend "remote" {
    hostname = "app.terraform.io"
    organization = "pphan"
    workspaces {
      name = "tg-demo-${path_relative_to_include()}"
    }
  }
}
EOF
}
```
  </details>

<details><summary><h3>Keep your CLI flags DRY</summary>

```go
terraform {
  # Force Terraform to keep trying to acquire a lock for
  # up to 20 minutes if someone else already has the lock
  extra_arguments "retry_lock" {
    commands = [
      "init",
      "apply",
      "refresh",
      "import",
      "plan",
      "taint",
      "untaint"
    ]

    arguments = [
      "-lock-timeout=20m"
    ]

    env_vars = {
      TF_VAR_var_from_environment = "value"
    }
  }
}
```
</details>

<details><summary><h3>Execute Terraform commands on multiple modules at once</h3></summary>

To be able to deploy multiple Terraform modules in a single command, add a `terragrunt.hcl` file to each module:

```text
root
├── backend-app
│   ├── main.tf
│   └── terragrunt.hcl
├── frontend-app
│   ├── main.tf
│   └── terragrunt.hcl
├── mysql
│   ├── main.tf
│   └── terragrunt.hcl
├── redis
│   ├── main.tf
│   └── terragrunt.hcl
└── vpc
    ├── main.tf
    └── terragrunt.hcl
```

<br>
Now, deploy all the modules within the `root` folder by using the `run-all` command with `apply`.:
<p>

```shell
cd root
terragrunt run-all apply
```

</details>

## Prep

### Install Terragrunt

In [ ]:
TERRAGRUNT_VERSION=0.38.8

Download, rename, make executable, and move it.

In [ ]:
curl -Lo /tmp/terragrunt \
  https://github.com/gruntwork-io/terragrunt/releases/download/v${TERRAGRUNT_VERSION}/terragrunt_linux_amd64
chmod +x /tmp/terragrunt
sudo mv /tmp/terragrunt /usr/local/bin/

Confirm

In [ ]:
terragrunt | grep -A2 -i version

### AWS Credentials

Set your AWS Credentials. I got one from Instruqt terminal with this command.

```bash
env | grep -iE "^aws.*access" | xargs -I{} echo export {}
```

In [ ]:
unset AWS_ACCESS_KEY_ID AWS_SECRET_ACCESS_KEY
export AWS_DEFAULT_REGION=us-west-2
export AWS_REGION=$AWS_DEFAULT_REGION
export AWS_ACCESS_KEY_ID=
export AWS_SECRET_ACCESS_KEY=
export TF_VAR_aws_access_key_id=$AWS_ACCESS_KEY_ID
export TF_VAR_aws_secret_access_key=$AWS_SECRET_ACCESS_KEY

printf "%s\n" "#==> Creds:" "$AWS_REGION" "$AWS_ACCESS_KEY_ID" "$AWS_SECRET_ACCESS_KEY"

Create default VPC if needed.

In [ ]:
aws configure set region us-west-2 --profile default
aws ec2 create-default-vpc > /dev/null || true
printf "\n#==> Show VPC ids\n"
aws ec2 describe-vpcs | jq -r '.[] | .[] | .VpcId'

# Example

Here is an example configuration you can use to get started. The following configuration can be used to deploy the `terraform-aws-modules/vpc` module from the Terraform Registry:

- In this demo we will run through terragrunt using a local backend.
- We will create a folder for creating TFC workspaces.
- We will create a folder for each environment (`dev`, `qa`, `prod`)
- All folders with `terragrunt.hcl` will be processed except the root.
- We will show module dependencies
    - workspace, dev, qa, and then prod
- Then, we will migrate all folders to TFC remote backend.
- Run all folders again with TFC to use TFC workers, variables, state, governance and more.

## Create folder structure

In [ ]:
mkdir -p /tmp/terragrunt/{workspace,dev,qa,prod}
touch /tmp/terragrunt/{workspace,dev,qa,prod}/terragrunt.hcl

In [ ]:
tree /tmp/terragrunt

Sample Output
```text
/tmp/terragrunt
├── dev
│   └── terragrunt.hcl
├── prod
│   └── terragrunt.hcl
├── qa
│   └── terragrunt.hcl
├── terragrunt.hcl
└── workspace
    ├── terragrunt.hcl
```

In [ ]:
cd /tmp/terragrunt

## create terragrunt.hcl - root

In [ ]:
cat > /tmp/terragrunt/terragrunt.hcl <<"EOL"
# Indicate where to source the terraform module from.
# The URL used here is a shorthand for
# "tfr://registry.terraform.io/terraform-aws-modules/vpc/aws?version=3.5.0".
# Note the extra `/` after the protocol is required for the shorthand
# notation.
terraform {
  source = "tfr:///terraform-aws-modules/ec2-instance/aws?version=4.0.0"
  before_hook "before_hook" {
    commands     = ["apply"]
    execute      = ["echo", "Applying my terraform"]
  }

  after_hook "after_hook" {
    commands     = ["apply"]
    execute      = ["echo", "Finished applying Terraform successfully!"]
    run_on_error = false
  }
  extra_arguments "automation" {
    commands = [
      "apply",
      "refresh",
      "taint",
      "untaint"
    ]
    arguments = [
      "-input=false",
      "-auto-approve"
    ]

    env_vars = {
      TF_VAR_var_from_environment = "value"
    }
  }
}

generate "versions" {
  path = "versions.tf"
  if_exists = "overwrite"
  contents = <<EOF
terraform {
  # required_version = "~1.1.0"
  required_providers{
    aws = {
      source = "hashicorp/aws"
      version = ">= 4.28.0"
    }
  }
}
EOF
}

generate "provider" {
  path = "provider.tf"
  if_exists = "overwrite_terragrunt"
  contents = <<EOF
provider "aws" {
  region = "us-west-2"   # region to deploy the resources into
}
EOF
}

# Indicate the input values to use for the variables of the module.
inputs = {
  ami           = "ami-06f29effee622eb00"
  instance_type = "t3.micro"
  tags = {
    Terraform   = "true"
    Environment = "root"
    Name        = "Terragrunt-${path_relative_to_include()}"
  }
}

generate "remote_state" {
  path      = "backend.tf"
  if_exists = "overwrite"
  contents = <<EOF
terraform {
  backend "remote" {
    hostname = "app.terraform.io" # Change this to your hostname for TFE
    organization = "pphan"
    workspaces {
      name = "tg-demo-${path_relative_to_include()}"
    }
  }
}
EOF
}

EOL

## create terragrunt.hcl - leaf workspace

In [ ]:
cat > /tmp/terragrunt/workspace/terragrunt.hcl <<"EOL"
terraform {
  extra_arguments "automation" {
    commands = [
      "apply",
      "refresh",
      "taint",
      "untaint"
    ]
    arguments = [
      "-input=false",
      "-auto-approve"
    ]

    env_vars = {
      TF_VAR_var_from_environment = "value"
    }
  }
}
#--> TF_VARs
inputs = {
  workspaces = ["dev","qa","prod"]
  tag_names  = ["test", "app"]
  tf_org_name = "pphan"
}

#--> Use this for testing different versions of providers.
generate "versions" {
  path = "versions.tf"
  if_exists = "overwrite"
  contents = <<EOF
terraform {
  required_providers{
    tfe = {
      source = "hashicorp/tfe"
      version = ">= 0.36.1"
    }
  }
}
EOF
}

#--> Enable this if TFC
generate "tfvars" {
  path      = "terragrunt.auto.tfvars.json"
  if_exists = "overwrite"
  disable_signature = true
  contents = jsonencode({ami = "ami-06f29effee622eb00",
      instance_type = "t3.micro",
      tags = {
        Terraform   = "true"
        Environment = "${path_relative_to_include()}"
        Name        = "Terragrunt-${path_relative_to_include()}"      
      }
  })
}

EOL

> **NOTE**: You will need to comment out the Run Tasks sections if you don't have it setup.

In [ ]:
cat > /tmp/terragrunt/workspace/workspace.tf <<"EOL"

resource "tfe_workspace" "this" {
  for_each     = toset(var.workspaces)
  name         = "tg-demo-${each.key}"
  description  = "terraform vending machine"
  organization = var.tf_org_name
  auto_apply   = true
  queue_all_runs = true
  terraform_version = "1.2.1"
  tag_names    = var.tag_names
}

resource "tfe_variable" "org_name" {
  for_each     = toset(var.workspaces)
  key   = "org_name"
  value = var.tf_org_name
  category = "terraform"
  workspace_id = tfe_workspace.this[each.key].id
  description = "TFC Org Name"
  sensitive = false
}

resource "tfe_variable_set" "this" {
  name          = "Test Varset"
  description   = "Terragrunt Variable Set"
  organization  = var.tf_org_name
}

resource "tfe_workspace_variable_set" "this" {
  for_each     = toset(var.workspaces)
  workspace_id    = tfe_workspace.this[each.key].id
  variable_set_id = tfe_variable_set.this.id
}

resource "tfe_variable" "aws_access_key_id" {
  key             = "AWS_ACCESS_KEY_ID"
  value           = var.aws_access_key_id
  category        = "env"
  description     = "a useful description"
  variable_set_id = tfe_variable_set.this.id
}

resource "tfe_variable" "aws_secret_access_key" {
  key             = "AWS_SECRET_ACCESS_KEY"
  value           = var.aws_secret_access_key
  category        = "env"
  description     = "a useful description"
  variable_set_id = tfe_variable_set.this.id
  sensitive       = true
}

data "tfe_organization_run_task" "infracost" {
  name         = "Infracost"
  organization = var.tf_org_name
}

resource "tfe_workspace_run_task" "infracost" {
  for_each          = toset(var.workspaces)
  workspace_id      = tfe_workspace.this[each.key].id
  task_id           = data.tfe_organization_run_task.infracost.id
  enforcement_level = "advisory"
}

data "tfe_organization_run_task" "snyk" {
  name         = "Snyk"
  organization = var.tf_org_name
}

resource "tfe_workspace_run_task" "snyk" {
  for_each          = toset(var.workspaces)
  workspace_id      = tfe_workspace.this[each.key].id
  task_id           = data.tfe_organization_run_task.snyk.id
  enforcement_level = "advisory"
}

variable "tf_org_name" {}

variable "workspaces" { type = list(string) }

variable "tag_names" { type = list(string) }

variable "aws_access_key_id" {}
variable "aws_secret_access_key" {}

output "workspaces_pros" {
  value = tfe_workspace.this
}

EOL

In [ ]:
pushd /tmp/terragrunt/workspace/
# terragrunt init
# terragrunt plan
terragrunt apply -auto-approve
# terragrunt destroy -auto-approve
popd

If an `input` variable has the same name as a variable defined in one of our shared variable `.yaml` files then it will be automatically picked up.

## create terragrunt.hcl - leaf dev

Tell the Terragrunt to import the root `terragrunt.hcl`. Add to `terragrunt.hcl` in leaf.

In [ ]:
cat > /tmp/terragrunt/dev/terragrunt.hcl <<"EOL"
include "root" {
  path = find_in_parent_folders()
}

dependencies {
  paths = ["../workspace"]
}

inputs = {
  ami           = "ami-06f29effee622eb00"
  instance_type = "t3.micro"
  tags = {
    Terraform   = "true"
    Environment = "${path_relative_to_include()}"
    Name        = "Terragrunt-${path_relative_to_include()}"
  }
}

#--> Use this for testing different versions of providers.
# generate "versions" {
#   path = "versions.tf"
#   if_exists = "overwrite" #"overwrite_terragrunt"
#   contents = <<EOF
# terraform {
#   # required_version = "~1.1.0"
#   required_providers{
#     aws = {
#       source = "hashicorp/aws"
#       version = ">= 4.28.0"
#     }
#   }
# }
# EOF
# }

#--> Enable this if TFC
generate "tfvars" {
  path      = "terragrunt.auto.tfvars.json"
  if_exists = "overwrite"
  disable_signature = true
  contents = jsonencode({ami = "ami-06f29effee622eb00",
      instance_type = "t3.micro",
      tags = {
        Terraform   = "true"
        Environment = "${path_relative_to_include()}"
        Name        = "Terragrunt-${path_relative_to_include()}"      
      }
  })
}

EOL

If an `input` variable has the same name as a variable defined in one of our shared variable `.yaml` files then it will be automatically picked up.

## create terragrunt.hcl - leaf qa

Tell the Terragrunt to import the root `terragrunt.hcl`. Add to `terragrunt.hcl` in leaf.

In [ ]:
cat > /tmp/terragrunt/qa/terragrunt.hcl <<"EOL"
include "root" {
  path = find_in_parent_folders()
}

dependencies {
  paths = ["../dev","../workspace"]
}

inputs = {
  ami           = "ami-06f29effee622eb00"
  instance_type = "t3.medium"
  tags = {
    Terraform   = "true"
    Environment = "${path_relative_to_include()}"
    Name        = "Terragrunt-${path_relative_to_include()}"
  }
}

#--> Use this for testing different versions of providers.
# generate "versions" {
#   path = "versions.tf"
#   if_exists = "overwrite" #"overwrite_terragrunt"
#   contents = <<EOF
# terraform {
#   # required_version = "~1.1.0"
#   required_providers{
#     aws = {
#       source = "hashicorp/aws"
#       version = ">= 4.28.0"
#     }
#   }
# }
# EOF
# }

#--> Enable this if TFC
generate "tfvars" {
  path      = "terragrunt.auto.tfvars.json"
  if_exists = "overwrite"
  disable_signature = true
  contents = jsonencode({ami = "ami-06f29effee622eb00",
      instance_type = "t3.medium",
      tags = {
        Terraform   = "true"
        Environment = "${path_relative_to_include()}"
        Name        = "Terragrunt-${path_relative_to_include()}"      
      }
  })
}

EOL

If an `input` variable has the same name as a variable defined in one of our shared variable `.yaml` files then it will be automatically picked up.

## create terragrunt.hcl - leaf prod

Tell the Terragrunt to import the root `terragrunt.hcl`. Add to `terragrunt.hcl` in leaf.

In [ ]:
cat > /tmp/terragrunt/prod/terragrunt.hcl <<"EOL"
include "root" {
  path = find_in_parent_folders()
}

dependencies {
  paths = ["../dev","../qa","../workspace"]
}

inputs = {
  ami           = "ami-06f29effee622eb00"
  instance_type = "t3.large"
  tags = {
    Terraform   = "true"
    Environment = "${path_relative_to_include()}"
    Name        = "Terragrunt-${path_relative_to_include()}"
  }
}

#--> Use this for testing different versions of providers.
# generate "versions" {
#   path = "versions.tf"
#   if_exists = "overwrite" #"overwrite_terragrunt"
#   contents = <<EOF
# terraform {
#   # required_version = "~1.1.0"
#   required_providers{
#     aws = {
#       source = "hashicorp/aws"
#       version = ">= 4.28.0"
#     }
#   }
# }
# EOF
# }

#--> Enable this if TFC
generate "tfvars" {
  path      = "terragrunt.auto.tfvars.json"
  if_exists = "overwrite"
  disable_signature = true
  contents = jsonencode({ami = "ami-06f29effee622eb00",
      instance_type = "t3.large",
      tags = {
        Terraform   = "true"
        Environment = "${path_relative_to_include()}"
        Name        = "Terragrunt-${path_relative_to_include()}"      
      }
  })
}

EOL

If an `input` variable has the same name as a variable defined in one of our shared variable `.yaml` files then it will be automatically picked up.

### walk thru

In the configuration,

- the `terraform` block is used to configure how Terragrunt will interact with Terraform.
    - You can configure things like `before` and `after` hooks for indicating custom commands to run before and after each terraform call, or what CLI args to pass in for each commands.
    - Here we only use it to indicate where terragrunt should fetch the terraform code using the `source` attribute.
    - We indicate that terragrunt should fetch the code from the `terraform-aws-modules/vpc/aws` module hosted in the Public Terraform Registry, version `3.5.0`.
    - This is indicated by using the `tfr://` protocol in the source URL, which takes the form:

    ```text
    tfr://REGISTRY_DOMAIN/MODULE?version=VERSION
    ```

    > Note that you can omit the `REGISTRY_DOMAIN` to default to the Public Terraform Registry.

- The `generate` block is used to inject the provider configuration into the active Terraform module.
    - This can be used to customize how Terraform interacts with the cloud APIs, including configuring authentication parameters.
    - `if_exists = "overwrite_terragrunt"` - overwrite if it exists

- The `inputs` block is used to indicate what variable values should be passed to terraform.
    - This is equivalent to having the contents of the map in a `tfvars` file and passing that to terraform.
    - this
        ```go
        inputs = {
          instance_type  = "t2.micro"
          instance_count = 10

          tags = {
            Name = "example-app"
          }
        }
        ```
    - becomes this
        ```go
        TF_VAR_instance_type="t2.micro" \
        TF_VAR_instance_count=10 \
        TF_VAR_tags='{"Name":"example-app"}'
        ```

You can read more about all the supported blocks of the terragrunt configuration in the [reference documentation](https://terragrunt.gruntwork.io/docs/reference/config-blocks-and-attributes), including additional sources that terragrunt supports.

You can deploy this example by copy pasting it into a folder and running `terragrunt apply`.

## terragrunt apply all - locally

**NOTE**: Heads up, not all Registry modules can be deployed with Terragrunt, see A note about using modules from the registry for details.

In [ ]:
cd /tmp/terragrunt

In [ ]:
# rm -rf /tmp/terragrunt/dev/.terragrunt-cache
terragrunt run-all init -force-copy \
  --terragrunt-exclude-dir . > /tmp/tf_tg_init_all_out.txt 2>&1 &

In [ ]:
tail /tmp/tf_tg_init_all_out.txt

Trigger run in TFC. Exclude parent folder.

In [ ]:
terragrunt run-all apply --terragrunt-non-interactive \
  --terragrunt-exclude-dir . \
  > /tmp/tf_tg_apply_all_out.txt 2>&1 &

Check progress in log file or in the UI.

In [ ]:
tail -n 50 /tmp/tf_tg_apply_all_out.txt

### Verify

1. Go to your console - https://instruqt-050659821008.signin.aws.amazon.com/console

2. Or verify from the CLI

In [ ]:
aws ec2 describe-instances \
  --query "Reservations[*].Instances[*].{IP:PublicIpAddress,Tags:Tags,InstanceType:InstanceType}" \
  | jq

# terragrunt output | grep -Ev '\"\"|\[\]'
# .{IP:PublicIpAddress,Tags:Tags,InstanceType:InstanceType}

# TFC and Terragrunt - Better Together

## Challenges around TF OSS that can be resolved by TFC/E

1. Credentials
2. State sharing/locking/queueing/ Authentication/Authorisation
3. Security and Operational governance(sentinel,runtask)
4. Cost estimation
5. Git workflow

## Challenges around TF OSS that is resolved by terragrunt/terratest

1. inputs/outputs validate
2. Test after tf apply

## Limitations of Terragrunt-Managed Backends

1. Terragrunt lacks security defaults on the log bucket
2. Disabling auto-creation of the state bucket and lock table is broken
3. Terragrunt doesn’t offer full control over the credentials used to access the Terraform state
4. Terragrunt doesn’t offer full control over all fields on the buckets and table

Source: https://www.bti360.com/advantages-and-limitations-of-terragrunt-managed-backends/

We can see that the challenges resolved by TFE/C is completely different with the challenges resolved by terragrant. Hence why it is a better together story.

## Migrating from terragrunt to TFC/E

1. Change generated backend
1. Populate the credentials
1. optional - Change the workflow from CLI to VCS


Changes required to support TFC.

1. Generate backend for TFC.
    - Uncomment `backend` section. See `/tmp/terragrunt/terragrunt.hcl`
1. Create `auto.tfvars` to support variables.
    - Uncomment `tfvars` section.
1. Populate the credentials
    - Configure Variables and/or Variable Sets
    - We created Variable Sets and applied them to Workspaces in `workspace`
1. Optional - These were applied in `workspace` folder
    1. Change Apply Method to `Auto apply`.
    1. Add Run Tasks
    1. Add Sentinel Policies

## terragrunt apply all - Switching to TFC

In [ ]:
cd /tmp/terragrunt

Migrate state file to TFC. Exclude parent folder.

Switching from local state to remote state requires running `terragrunt init`. We add `-force-copy` to have it not prompt us.

In [ ]:
# rm -rf /tmp/terragrunt/dev/.terragrunt-cache
terragrunt run-all init -force-copy \
  --terragrunt-exclude-dir .

`terragrunt run-all apply`. Things to note:
- First, `dev` will will run first, since we stated that `prod` has a dependency on it.
- Then, `prod` will run.
    - If dependency was not then all will run concurrently.

In [ ]:
terragrunt run-all apply --terragrunt-non-interactive \
  --terragrunt-exclude-dir . \
  > /tmp/tf_tg_apply_all_out.txt 2>&1 &

Check progress.

In [ ]:
tail -n 50 /tmp/tf_tg_apply_all_out.txt

### Benefits 

- the state is now centrally stored, with RBAC.
    - The credential is centrally controlled, no one can read it.
- In addition to this, you can then add sentinel or run task to show guardrail.

# Key features

Terragrunt can help you accomplish the following:

- Keep your backend configuration DRY
- Keep your provider configuration DRY
- Keep your Terraform CLI arguments DRY
- Promote immutable, versioned Terraform modules across environments

## Keep your backend configuration DRY

Terraform backends allow you to store Terraform state in a shared location that everyone on your team can access, such as an S3 bucket, and provide locking around your state files to protect against race conditions. To use a Terraform backend, you add a backend configuration to your Terraform code:

```go
#stage/frontend-app/main.tf
terraform {
  backend "s3" {
    bucket         = "my-terraform-state"
    key            = "stage/frontend-app/terraform.tfstate"
    region         = "us-east-1"
    encrypt        = true
    dynamodb_table = "my-lock-table"
  }
}
```

The code above tells Terraform to store the state for a `frontend-app` module in an S3 bucket called `my-terraform-state` under the path `stage/frontend-app/terraform.tfstate`, and to use a DynamoDB table called `my-lock-table` for locking. This is a great feature that every single Terraform team uses to collaborate, but it comes with one major gotcha: the backend configuration does not support variables or expressions of any sort. That is, the following will NOT work:

```go
#stage/frontend-app/main.tf
terraform {
  backend "s3" {
    # Using variables does NOT work here!
    bucket         = var.terraform_state_bucket
    key            = var.terraform_state_key
    region         = var.terraform_state_region
    encrypt        = var.terraform_state_encrypt
    dynamodb_table = var.terraform_state_dynamodb_table
  }
}
```

That means you have to copy/paste the same `backend` configuration into every one of your Terraform modules. Not only do you have to copy/paste, but you also have to very carefully not copy/paste the `key` value so that you don’t have two modules overwriting each other’s state files! E.g., The `backend` configuration for a `database` module would look nearly identical to the `backend` configuration of the `frontend-app` module, except for a different `key` value:

```go
#stage/mysql/main.tf
terraform {
  backend "s3" {
    bucket         = "my-terraform-state"
    key            = "stage/mysql/terraform.tfstate"
    region         = "us-east-1"
    encrypt        = true
    dynamodb_table = "my-lock-table"
  }
}
```

Terragrunt allows you to keep your `backend` configuration DRY (“Don’t Repeat Yourself”) by defining it once in a root location and inheriting that configuration in all child modules. Let’s say your Terraform code has the following folder layout:

```
stage
├── frontend-app
│   └── main.tf
└── mysql
    └── main.tf
```

To use Terragrunt, add a single `terragrunt.hcl` file to the root of your repo, in the `stage` folder, and one `terragrunt.hcl` file in each module folder:

```text
stage
├── terragrunt.hcl
├── frontend-app
│   ├── main.tf
│   └── terragrunt.hcl
└── mysql
    ├── main.tf
    └── terragrunt.hcl
```

Now you can define your `backend` configuration just once in the root `terragrunt.hcl` file:

```text
#stage/terragrunt.hcl
remote_state {
  backend = "s3"
  generate = {
    path      = "backend.tf"
    if_exists = "overwrite_terragrunt"
  }
  config = {
    bucket = "my-terraform-state"

    key = "${path_relative_to_include()}/terraform.tfstate"
    region         = "us-east-1"
    encrypt        = true
    dynamodb_table = "my-lock-table"
  }
}
```

The `terragrunt.hcl` files use the same configuration language as Terraform (HCL) and the configuration is more or less the same as the `backend` configuration you had in each module, except that the key value is now using the `path_relative_to_include()` built-in function, which will automatically set key to the relative path between the root `terragrunt.hcl` and the child module (so your Terraform state folder structure will match your Terraform code folder structure, which makes it easy to go from one to the other).

The `generate` attribute is used to inform Terragrunt to generate the Terraform code for configuring the backend. When you run any Terragrunt command, Terragrunt will generate a `backend.tf` file with the contents set to the terraform block that configures the s3 backend, just like what we had before in each `main.tf` file.

The final step is to update each of the child `terragrunt.hcl` files to tell them to include the configuration from the root `terragrunt.hcl`:

```text
#stage/mysql/terragrunt.hcl
include "root" {
  path = find_in_parent_folders()
}
```

The `find_in_parent_folders()` helper will automatically search up the directory tree to find the root `terragrunt.hcl` and inherit the `remote_state` configuration from it.

Now, install Terragrunt, and run all the Terraform commands you’re used to, but with `terragrunt` as the command name rather than `terraform` (e.g., `terragrunt apply` instead of `terraform apply`). To deploy the database module, you would run:

```shell
$ cd stage/mysql
$ terragrunt apply
```

Terragrunt will automatically find the mysql module’s `terragrunt.hcl` file, configure the backend using the settings from the root `terragrunt.hcl` file, and, thanks to the `path_relative_to_include()` function, will set the key to `stage/mysql/terraform.tfstate`. If you run `terragrunt apply` in `stage/frontend-app`, it’ll do the same, except it will set the key to `stage/frontend-app/terraform.tfstate`.

You can now add as many child modules as you want, each with a `terragrunt.hcl` with the `include "root" { … }` block, and each of those modules will automatically inherit the proper backend configuration!

## Keep your provider configuration DRY

Unifying provider configurations across all your modules can be a pain, especially when you want to customize authentication credentials. To configure Terraform to assume an IAM role before calling out to AWS, you need to add a `provider` block with the `assume_role` configuration:

```go
#--> stage/frontend-app/main.tf
provider "aws" {
  assume_role {
    role_arn = "arn:aws:iam::0123456789:role/terragrunt"
  }
}
```

This code tells Terraform to assume the `role arn:aws:iam::0123456789:role/terragrunt` prior to calling out to the AWS APIs to create the resources. Unlike the `backend` configurations, `provider` configurations support variables, so typically you will resolve this by making the role configurable in the module:

```go
#--> stage/frontend-app/main.tf
variable "assume_role_arn" {
  description = "Role to assume for AWS API calls"
}

provider "aws" {
  assume_role {
    role_arn = var.assume_role_arn
  }
}
```

You would then copy paste this configuration in every one of your Terraform modules. This isn’t a lot of lines of code, but can be a pain to maintain. For example, if you needed to modify the configuration to expose another parameter (e.g `session_name`), you would have to then go through each of your modules to make this change.

In addition, what if you wanted to directly deploy a general purpose module, such as that from the [Terraform module registry](https://registry.terraform.io/) or the [Gruntwork Infrastructure as Code library](https://gruntwork.io/infrastructure-as-code-library/)? These modules typically do not expose provider configurations as it is tedious to expose every single provider configuration parameter imaginable through the module interface.

Terragrunt allows you to refactor common Terraform code to keep your Terraform modules DRY. Just like with the `backend` configuration, you can define the `provider` configurations once in a root location. In the root `terragrunt.hcl` file, you would define the provider configuration using the `generate` block:

```go
#--> stage/terragrunt.hcl
generate "provider" {
  path = "provider.tf"
  if_exists = "overwrite_terragrunt"
  contents = <<EOF
provider "aws" {
  assume_role {
    role_arn = "arn:aws:iam::0123456789:role/terragrunt"
  }
}
EOF
}
```

This instructs Terragrunt to create the file `provider.tf` in the working directory (where Terragrunt calls `terraform`) before it calls any of the Terraform commands (e.g `plan`, `apply`, `validate`, etc). This allows you to inject this provider configuration in all the modules that includes the root file without having to define them in the underlying modules.

When you run `terragrunt plan` or `terragrunt apply`, you can see that this file is created in the module working directory:

```shell
$ cd stage/mysql
$ terragrunt apply
$ find . -name "provider.tf"
.terragrunt-cache/some-unique-hash/provider.tf
```

In [ ]:
cd /tmp/terragrunt/dev
# terragrunt plan
find . -name "provider.tf"

Sample Output
```text
./.terragrunt-cache/uN5OMu9iL7TZWfc9R2CduOixZFQ/pfgqyj3TsBEWff7a1El6tYu6LEE/provider.tf
```

## Keep your Terraform CLI arguments DRY

CLI flags are another common source of copy/paste in the Terraform world. For example, a typical pattern with Terraform is to define common account-level variables in an `account.tfvars` file:

```
# account.tfvars
account_id     = "123456789012"
account_bucket = "my-terraform-bucket"
```

And to define common region-level variables in a `region.tfvars` file:

```
# region.tfvars
aws_region = "us-east-2"
foo        = "bar"
```
You can tell Terraform to use these variables using the `-var-file` argument:

```shell
$ terraform apply \
    -var-file=../../common.tfvars \
    -var-file=../region.tfvars

```

> Having to remember these `-var-file` arguments every time can be tedious and error prone.

Terragrunt allows you to keep your CLI arguments DRY by defining those arguments as code in your `terragrunt.hcl` configuration:

```go
# terragrunt.hcl
terraform {
  extra_arguments "common_vars" {
    commands = ["plan", "apply"]

    arguments = [
      "-var-file=../../common.tfvars",
      "-var-file=../region.tfvars"
    ]
  }
}
```

Now, when you run the `plan` or `apply` commands, Terragrunt will automatically add those arguments:

```shell
$ terragrunt apply

Running command: terraform with arguments
[apply -var-file=../../common.tfvars -var-file=../region.tfvars]
```

You can even use the `get_terraform_commands_that_need_vars()` built-in function to automatically get the list of all commands that accept `-var-file` and `-var` arguments:

```
# terragrunt.hcl
terraform {
  extra_arguments "common_vars" {
    commands = get_terraform_commands_that_need_vars()

    arguments = [
      "-var-file=../../common.tfvars",
      "-var-file=../region.tfvars"
    ]
  }
}
```

## Promote immutable, versioned Terraform modules across environments

One of the most important [lessons we’ve learned from writing hundreds of thousands of lines of infrastructure code](https://blog.gruntwork.io/5-lessons-learned-from-writing-over-300-000-lines-of-infrastructure-code-36ba7fadeac1) is that large modules should be considered harmful. That is, it is a Bad Idea to define all of your environments (dev, stage, prod, etc), or even a large amount of infrastructure (servers, databases, load balancers, DNS, etc), in a single Terraform module. Large modules are slow, insecure, hard to update, hard to code review, hard to test, and brittle (i.e., you have all your eggs in one basket).

Therefore, you typically want to break up your infrastructure across multiple modules:

  ```text
  ├── prod
  │   ├── app
  │   │   ├── main.tf
  │   │   └── outputs.tf
  │   ├── mysql
  │   │   ├── main.tf
  │   │   └── outputs.tf
  │   └── vpc
  │       ├── main.tf
  │       └── outputs.tf
  ├── qa
  │   ├── app
  │   │   ├── main.tf
  │   │   └── outputs.tf
  │   ├── mysql
  │   │   ├── main.tf
  │   │   └── outputs.tf
  │   └── vpc
  │       ├── main.tf
  │       └── outputs.tf
  └── stage
      ├── app
      │   ├── main.tf
      │   └── outputs.tf
      ├── mysql
      │   ├── main.tf
      │   └── outputs.tf
      └── vpc
          ├── main.tf
          └── outputs.tf
  ```

The folder structure above shows how to separate the code for each environment (`prod`, `qa`, `stage`) and for each type of infrastructure (`apps`, `databases`, `VPCs`). However, the downside is that it isn’t DRY. The `.tf` files will contain a LOT of duplication. You can reduce it somewhat by defining all the infrastructure in [reusable Terraform modules](https://blog.gruntwork.io/how-to-create-reusable-infrastructure-with-terraform-modules-25526d65f73d), but even the code to instantiate a module - including configuring the provider, backend, the module’s input variables, and `output` variables - means you still end up with dozens or hundreds of lines of copy/paste for every module in every environment:

```go
# prod/app/main.tf
provider "aws" {
  region = "us-east-1"
  # ... other provider settings ...
}
terraform {
  backend "s3" {}
}
module "app" {
  source = "../../../app"
  instance_type  = "m4.large"
  instance_count = 10
  # ... other app settings ...
}
# prod/app/outputs.tf
output "url" {
  value = module.app.url
}
# ... and so on!
```

Terragrunt allows you to define your Terraform code once and to promote a versioned, immutable “artifact” of that exact same code from environment to environment. Here’s a quick overview of how.

First, create a Git repo called `infrastructure-modules` that has your Terraform code (`.tf` files). This is the exact same Terraform code you just saw above, except that any variables that will differ between environments should be exposed as input variables:

```go 
#--> infrastructure-modules/app/main.tf
provider "aws" {
  region = "us-east-1"
  # ... other provider settings ...
}
terraform {
  backend "s3" {}
}
module "app" {
  source = "../../../app"
  instance_type  = var.instance_type
  instance_count = var.instance_count
  # ... other app settings ...
}
#--> infrastructure-modules/app/outputs.tf
output "url" {
  value = module.app.url
}
#--> infrastructure-modules/app/variables.tf

variable "instance_type" {}
variable "instance_count" {}
```

Once this is in place, you can release a new version of this module by creating a Git tag:

```shell
$ git tag -a "v0.0.1" -m "First release of app module"
$ git push --follow-tags
```

Now, in another Git repo called `infrastructure-live`, you create the same folder structure you had before for all of your environments, but instead of lots of copy/pasted `.tf` files for each module, you have just a single `terragrunt.hcl` file:

```text
#--> infrastructure-live
├── prod
│   ├── app
│   │   └── terragrunt.hcl
│   ├── mysql
│   │   └── terragrunt.hcl
│   └── vpc
│       └── terragrunt.hcl
├── qa
│   ├── app
│   │   └── terragrunt.hcl
│   ├── mysql
│   │   └── terragrunt.hcl
│   └── vpc
│       └── terragrunt.hcl
└── stage
    ├── app
    │   └── terragrunt.hcl
    ├── mysql
    │   └── terragrunt.hcl
    └── vpc
        └── terragrunt.hcl
```

The contents of each `terragrunt.hcl` file look something like this:

```go
#--> infrastructure-live/prod/app/terragrunt.hcl
terraform {
  source =
    "github.com:foo/infrastructure-modules.git//app?ref=v0.0.1"
}
inputs = {
  instance_count = 10
  instance_type  = "m4.large"
}
```

The `terragrunt.hcl` file above sets the `source` parameter to point at the `app` module you just created in your infrastructure-modules repo, using the `ref` parameter to specify version `v0.0.1` of that repo. It also configures the variables for this module for the `prod` environment in the `inputs = {…​}` block.

The `terragrunt.hcl` file in the `stage` environment will look similar, but it will configure smaller/fewer instances in the `inputs = {…​}` block to save money:

```shell
#--> infrastructure-live/stage/app/terragrunt.hcl
terraform {
  source =
    "github.com:foo/infrastructure-modules.git//app?ref=v0.0.1"
}
inputs = {
  instance_count = 3
  instance_type  = "t2.micro"
}
```

When you run `terragrunt apply`, Terragrunt will download your app module into a temporary folder, run `terraform apply` in that folder, passing the module the input variables you specified in the `inputs = {…​}` block:

```shell
$ terragrunt apply
Downloading Terraform configurations from github.com:foo/infrastructure-modules.git...
Running command: terraform with arguments [apply]...
```

This way, each module in each environment is defined by a single `terragrunt.hcl` file that solely specifies the Terraform module to deploy and the input variables specific to that environment. This is about as DRY as you can get!

Moreover, you can specify a different version of the module to deploy in each environment! For example, after making some changes to the `app` module in the `infrastructure-modules` repo, you could create a `v0.0.2` tag, and update just the `qa` environment to run this new version:

```
#--> infrastructure-live/qa/app/terragrunt.hcl
terraform {
  source =
    "github.com:foo/infrastructure-modules.git//app?ref=v0.0.2"
}
inputs = {
  instance_count = 3
  instance_type  = "t2.micro"
}
```

If it works well in the `qa` environment, you could promote the exact same code to the `stage` environment by updating its `terragrunt.hcl` file to run `v0.0.2`. And finally, if that code works well in `stage`, you could again promote the exact same code to prod by updating that `terragrunt.hcl` file to use `v0.0.2` as well.

Using Terragrunt to promote immutable Terraform code across environments

If at any point you hit a problem, it will only affect the one environment, and you can roll back by deploying a previous version number. That’s immutable infrastructure at work!

# Optional - run individual folders

## terragrunt apply - workspace

In [ ]:
cd /tmp/terragrunt/workspace

In [ ]:
terragrunt init #-force-copy

In [ ]:
terragrunt plan > /tmp/tf_plan_workspace_out.txt 2>&1 &

In [ ]:
tail -n 50 /tmp/tf_plan_workspace_out.txt
# grep type /tmp/tf_plan_workspace_out.txt

In [ ]:
terragrunt apply > /tmp/tf_apply_workspace_out.txt 2>&1 &

In [ ]:
tail -n 50 /tmp/tf_apply_workspace_out.txt

## terragrunt apply - dev

In [ ]:
cd /tmp/terragrunt/dev

In [ ]:
terragrunt init #-force-copy

In [ ]:
terragrunt plan > /tmp/tf_plan_dev_out.txt 2>&1 &

In [ ]:
tail -n 50 /tmp/tf_plan_dev_out.txt
# grep type /tmp/tf_plan_dev_out.txt

### view files - optional

In [ ]:
pushd /tmp/terragrunt/dev > /dev/null
printf "%s\n%s\n\n" "VERSIONS:" "$(cat ./.terragrunt-cache/*/*/versions.tf)" \
  "BACKEND" "$(cat ./.terragrunt-cache/*/*/backend.tf)" \
  "PROVIDER" "$(cat ./.terragrunt-cache/*/*/provider.tf)" \
  "TFVARS" "$(cat ./.terragrunt-cache/*/*/*.json)"
popd > /dev/null

find /tmp/terragrunt -name "*.tfvars.json"

In [ ]:
terragrunt apply -auto-approve > tf_apply_dev_out.txt 2>&1 &

In [ ]:
tail -n 50 tf_apply_dev_out.txt

### Verify

1. Go to your console - https://instruqt-050659821008.signin.aws.amazon.com/console

2. Or verify from the CLI

In [ ]:
aws ec2 describe-vpcs | jq -r '.[] | .[] | .VpcId'

aws ec2 describe-subnets \
  --filter Name=vpc-id,Values=$(terragrunt output -raw vpc_id) \
  | jq -r ".Subnets[].SubnetId"

terragrunt output | grep -Ev '\"\"|\[\]'

## terragrunt-cache

UNDER DEVELOPMENT

I'm using this section to explore what terragrunt does behind the scenes.

In [ ]:
tree .terragrunt-cache

In [ ]:
cat .terragrunt-cache/ij_KdmoI4U-1gNbuOnp6ME0T4-E/pfgqyj3TsBEWff7a1El6tYu6LEE/versions.tf

In [ ]:
ls -al .terragrunt-cache/ij_KdmoI4U-1gNbuOnp6ME0T4-E/ThyYwttwki6d6AS3aD5OwoyqIWA/

In [ ]:
cat .terragrunt-cache/ij_KdmoI4U-1gNbuOnp6ME0T4-E/ThyYwttwki6d6AS3aD5OwoyqIWA/variables.tf

In [ ]:
ls -al

# Clean Up

## Destroy Workloads

Destroy `dev`, `qa`, `prod` workloads.

In [ ]:
pushd /tmp/terragrunt 
terragrunt run-all destroy --terragrunt-non-interactive \
  --terragrunt-exclude-dir . \
  --terragrunt-exclude-dir ./workspace \
  > /tmp/tf_tg_destroy_all_out.txt 2>&1 &
popd

In [ ]:
tail -n 50 /tmp/tf_tg_destroy_all_out.txt

## Destroy workspace

In [ ]:
pushd /tmp/terragrunt/workspace
terragrunt destroy -auto-approve > tf_destroy_workspace_out.txt 2>&1 &
popd

In [ ]:
tail /tmp/terragrunt/workspace/tf_destroy_workspace_out.txt

## Destroy entire terragrunt root folder

In [ ]:
rm -rf /tmp/terragrunt/

# Resources

- !! How to deploy production-grade infrastructure in a fraction of the time using Gruntwork with Terraform Cloud and Terraform Enterprise
    - !! https://blog.gruntwork.io/how-deploy-production-grade-infrastructure-using-gruntwork-with-terraform-cloud-aca919ca92c2
    - !! https://stackoverflow.com/questions/60062705/can-i-use-terragrunt-on-terraform-cloud
- https://terragrunt.gruntwork.io/docs/getting-started/quick-start/
- https://terragrunt.gruntwork.io/docs/reference/config-blocks-and-attributes/
- https://www.padok.fr/en/blog/terraform-code-terragrunt
- https://developer.newrelic.com/terraform/terragrunt-configuration/
- !! Terraform Code Layout and Using Terragrunt
    - https://medium.com/@AaronKalair/terraform-code-layout-and-using-terragrunt-db6864967916
- https://github.com/paddymorgan84/terragrunt-tutorial/tree/terragrunt
- https://dev.to/paddymorgan84/enhancing-terraform-with-terragrunt-540o
- !! Advantages and Limitations of Terragrunt-Managed Backends
    - https://www.bti360.com/advantages-and-limitations-of-terragrunt-managed-backends/
    - https://thirstydeveloper.io/tf-skeleton/2021/01/28/part-3-aws-backend.html

- Hashi Only: https://docs.google.com/presentation/d/1DXYLwrldFzGfPhSb6twgtMyNdIK-QNgd4O20B1KfO-U/edit